In [3]:
# Importations

import numpy as np # Pour les calculs mathématiques
import matplotlib.pyplot as plt # Pour les graphiques
import pandas as pd # Pour la manipulation de tableaux de données
from spellchecker import SpellChecker # Pour vérifier que le dataset d'entraînement ne contient pas de fautes d'orthographe ni de noms propres


ModuleNotFoundError: No module named 'indexer'

# Nettoyage de la base de données

In [ ]:
# Ouverture du fichier

file_path = '/Users/quentingirard/Desktop/Ecole/2A_Ponts/S4/PRAMA/europarl-v6.fr-en.fr_toy'     #Chemin de la BDD jouet pour Quentin
#file_path = '/Users/quentingirard/Desktop/Ecole/2A_Ponts/S4/PRAMA/europarl-v6.fr-en.fr_real'     #Chemin de la BDD complète pour Quentin

file = open(file_path, 'r', encoding='utf-8')
text = file.read()

In [ ]:
def remove_accents(text):
    text = text.replace('à', 'a')
    text = text.replace('â', 'a')
    text = text.replace('é', 'e')
    text = text.replace('è', 'e')
    text = text.replace('ê', 'e')
    text = text.replace('î', 'i')
    text = text.replace('ô', 'o')
    text = text.replace('ù', 'u')
    text = text.replace('û', 'u')
    text = text.replace('ç', 'c')
    return text

def remove_special_chars(text):
    text = text.replace('.', ' ')
    text = text.replace('!', ' ')
    text = text.replace('?', ' ')
    text = text.replace("'", ' ')
    text = text.replace('"', ' ')
    text = text.replace(',', ' ')
    text = text.replace(';', ' ')
    text = text.replace('(', '')
    text = text.replace(')','')

    return text

def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [ ]:
text = text.lower() # On met tout en minuscule - abandonné car on a besoin des majuscules pour identifier les noms propres.
#text = remove_accents(text) # On enlève les accents
text = remove_special_chars(text) # On remplace les caractères spéciaux par des espaces
text = remove_numbers(text) # On enlève les chiffres
print(text[1000:2000])

 est parvenu à interpréter et à respecter les différentes sensibilités présentes à la commission  y compris celles qui étaient de nature éthique 
pour ma part  en tant qu  homme de sciences  je suis extrêmement satisfait tant de la valeur intrinsèque particulière du rapport  qui place enfin l  europe dans les rangs des pays qui investissent le plus dans la recherche scientifique  que des nouvelles frontières qui s  ouvrent à la lutte contre des maladies terrifiantes  telles que certaines maladies dégénératives du système nerveux qui agissent sur le système nerveux central 
en outre  dans le domaine agroalimentaire  l  approbation d  un de mes amendements  qui prévoit que  pour les aliments transgéniques  avant leur introduction sur le marché  des tests de mutagénèse  d oncogénèse et de toxicité soient effectués obligatoirement  va dans la bonne direction pour gagner à nouveau la confiance des citoyens  particulièrement en ce qui concerne les institutions et le monde politique 
je remer

In [ ]:
## Vérification de la bonne orthographe des mots

spell = SpellChecker(language='fr')

words = text.split()
misspelled = spell.unknown(words)

list_misspelled = [word for word in misspelled if len(word) > 1]
print(list_misspelled)

['monicault', 'liikanen', 'vux', 'mutagénèse', 'europe', 'purvis', 'john', 'agroalimentaire', 'transgéniques', 'busquin', 'girard', 'oncogénèse', 'dégénératives']


In [ ]:
# Vérification de la bonne identification des mots mal orthographiés et des noms propres
misspelled_examples = ['ponnambalam','kumar','mutagénèse','busquin','tchernobyl']   #Attention, cette liste n'est pertinente que pour le dataset complet, et pas pour l'exemple jouet.
for word in misspelled_examples:
    print(word in list_misspelled)

False
False
True
True
False


Alors, on a deux choix : on peut enlever les mots mal orthographiés ou on peut les remplacer par une version corrigée. 

In [ ]:
percentage_misspelled = round(len(list_misspelled) / len(words) * 100,2)
print(f"Percentage of misspelled words: {percentage_misspelled:.2f}%")

Percentage of misspelled words: 3.49%


Le plus simple est de drop ces mots. On s'affranchit à la fois des noms propres et des mots mal orthographiés. Un modèle plus fin pourra être réalisé par la suite car supprimer les mots mal orthographiés nuit au sens des phrases.

On conserve les titres (ex :  M. Bonnefoi -> M est conservé).

In [ ]:
# On supprime les mots mal orthographiés et les noms propres

print(f'Nouvelle longueur du texte: {len(text.split())} mots')
for word in words:
    if word in list_misspelled:
        text = text.replace(word, '')

#print(text[1000:2000])
number_words = len(text.split())
print(f'Nouvelle longueur du texte: {number_words} mots')

Nouvelle longueur du texte: 373 mots
Nouvelle longueur du texte: 359 mots


Je n'arrive pas à faire compiler la cellule ci-dessus pour le vrai dataset (ça tourne pendant des heures, je ne sais pas quoi utiliser pour améliorer la complexité de la fonction).

# Création du dataset d'entraînement à partir de la base de données

In [ ]:
# Initialisation d'un dictionnaire
word_counts = {}

# Comptage des occurences des mots et remplissage du dictionnaire
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()
    words = text.split()
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

# Conversion du dictionnaire en DataFrame
df = pd.DataFrame(list(word_counts.items()), columns=['mot', 'occurence'])
df.set_index('mot', inplace=True)
df['fréquence (%)'] = df['occurence'] / number_words * 100

print(word_counts)
df.head()

{'Reprise': 1, 'de': 19, 'la': 8, 'session': 2, 'Je': 4, 'déclare': 1, 'reprise': 1, 'du': 3, 'Parlement': 1, 'européen': 1, 'qui': 11, 'avait': 1, 'été': 4, 'interrompue': 1, 'le': 10, 'vendredi': 1, '17': 1, 'décembre': 1, 'dernier': 1, 'et': 8, 'je': 3, 'vous': 6, 'renouvelle': 1, 'tous': 1, 'mes': 2, 'vux': 1, 'en': 3, 'espérant': 1, 'que': 4, 'avez': 3, 'passé': 1, 'bonnes': 1, 'vacances.': 1, 'Comme': 1, 'pu': 1, 'constater,': 1, 'grand': 1, '"bogue': 1, "l'an": 1, '2000"': 1, 'ne': 1, "s'est": 1, 'pas': 1, 'produit.': 1, 'En': 3, 'revanche,': 1, 'les': 8, 'citoyens': 1, "d'un": 1, 'certain': 2, 'nombre': 2, 'nos': 1, 'pays': 3, 'ont': 4, 'victimes': 1, 'catastrophes': 1, 'naturelles': 1, 'vraiment': 1, 'terribles.': 1, 'Vous': 1, 'souhaité': 1, 'un': 5, 'débat': 1, 'à': 7, 'ce': 3, 'sujet': 1, 'dans': 7, 'prochains': 1, 'jours,': 1, 'au': 1, 'cours': 1, 'cette': 2, 'période': 1, 'session.': 1, 'attendant,': 1, 'souhaiterais,': 1, 'comme': 1, 'collègues': 1, 'me': 1, "l'ont": 1, 

,occurence,fréquence (%)
mot,,
Reprise,1,0.278552
de,19,5.292479
la,8,2.228412
session,2,0.557103
Je,4,1.114206
